# utils

## excels
> excel工具类

In [1]:
from utils.excels import save_excel

In [2]:
import pandas as pd

inner_dict = {"name": "tobin", "id": 123}
test_dict = {123: inner_dict}

test_df = pd.DataFrame(test_dict)
save_excel(test_df.T, "/Users/tobin/Documents/Programs/datastory/tools/test_save_excel.xlsx")

# start at 1559668529.11158
save size: (1, 2)
# end at 1559668529.16523
total cost: 0.05365


In [3]:
from utils.excels import each_row

In [4]:
topic_user_file='/Users/tobin/Documents/Programs/datastory/tools/analysis_task_20190604193137_956_58.xlsx'
topic_user_sheet = pd.read_excel(topic_user_file, sheet_name='模板')

In [9]:

topic_user_infos = {}
def user_parser(idx, key, row):
    uid = row['uid']
    topic_user_infos[uid] = row
    
    return True

each_row(topic_user_sheet, user_parser, 1000)

# start at 1559668744.59189
current time: 1559668744.696231
current idx: 1000
current time: 1559668744.757088
current idx: 2000
current time: 1559668744.821700
current idx: 3000
current time: 1559668744.927420
current idx: 4000
current time: 1559668744.983626
current idx: 5000
# end at 1559668745.00677
total cost: 0.41488


In [10]:
print(len(topic_user_infos))

5446


## elasticsearchs

> ES操作工具类

In [1]:
from elasticsearch import Elasticsearch

proj_es_17 = Elasticsearch([{'host':'proj1', 'port':9200}])

1. 索引操作

In [2]:
from utils.elasticsearchs import create_index, delete_index

In [3]:
index = "py-index-v7"

In [5]:
# print(delete_index(proj_es_17, index))

In [6]:
fields = [
    "id|short",
    "name|string|words",
    "desc|string|not_analyzed",
    "card_num|long"
]
doc_type = "student"

In [7]:
print(create_index(proj_es_17, index, doc_type, fields))

index body: {
  "mappings": {
    "student": {
      "dynamic": "strict",
      "properties": {
        "card_num": {
          "type": "long"
        },
        "desc": {
          "doc_values": true,
          "index": "not_analyzed",
          "type": "string"
        },
        "id": {
          "type": "short"
        },
        "name": {
          "analyzer": "words",
          "type": "string"
        }
      }
    }
  },
  "settings": {
    "number_of_replica": 1,
    "number_of_shards": 5
  }
}
None


2. 数据操作

In [8]:
from utils.elasticsearchs import insert_one, insert_bulk

In [9]:
data_one = {
    "id": 587,
    "name": "张高峰",
    "desc": "任意用于测试不分词字段",
    "card_num": 8755427754613
}

print(insert_one(proj_es_17, index, doc_type, 587, data_one))

{'_index': 'py-index-v7', '_type': 'student', '_id': '587', '_version': 1, 'created': True}


In [10]:
data_bulk = {}
for i in range(13):
    temp = {
        "id": 587 + i,
        "name": "张高峰_%d" % i,
        "desc": "任意用于测试不分词字段",
        "card_num": 8755427754613 * i
    }
    if i == 10:
        temp["card"] = "故意检查动态字段问题"

    data_bulk[i + 1] = temp
    
print(len(data_bulk))

13


In [11]:
success, info = insert_bulk(proj_es_17, index, doc_type, data_bulk)

action size: 13


In [12]:
print(success)
print(info)

12
[{'index': {'_index': 'py-index-v7', '_type': 'student', '_id': '11', 'status': 400, 'error': 'StrictDynamicMappingException[mapping set to strict, dynamic introduction of [card] within [student] is not allowed]'}}]


3. 查询数据

In [1]:
import elasticsearch2

# 低版本查询专用client
proj_es_17 = elasticsearch2.Elasticsearch([{'host':'proj1', 'port':9200}], timeout=300)

In [2]:
from utils.elasticsearchs import scroll_parse

In [3]:
query = {
  "query": {
    "query_string": {
      "default_field": "0104dazhong-v1.country",
      "query": "singapore"
    }
  }
}

es_index = "0104dazhong-v1"
es_type = "0104dazhong-v1"

In [4]:
total_rows = {}

def row_parser(idx, key, value):
#     print(idx)
    total_rows[key] = value["_source"]
    return True
    
scroll_parse(proj_es_17, es_index, es_type, query, row_parser)

# start at 1559986569.76067
parsing: 0104dazhong-v1/0104dazhong-v1, batch size: 5, query: {
  "query": {
    "query_string": {
      "default_field": "0104dazhong-v1.country",
      "query": "singapore"
    }
  }
}
# current size: 30, progress: 30/749. At 1559986571.19044
# current size: 30, progress: 60/749. At 1559986572.92418
# current size: 30, progress: 90/749. At 1559986575.55932
# current size: 30, progress: 120/749. At 1559986578.15276
# current size: 30, progress: 150/749. At 1559986581.48203
# current size: 30, progress: 180/749. At 1559986584.75984
# current size: 30, progress: 210/749. At 1559986587.51502
# current size: 30, progress: 240/749. At 1559986589.36428
# current size: 30, progress: 270/749. At 1559986594.58723
# current size: 30, progress: 300/749. At 1559986599.91467
# current size: 30, progress: 330/749. At 1559986602.28495
# current size: 30, progress: 360/749. At 1559986606.67221
# current size: 30, progress: 390/749. At 1559986608.31394
# current size: 30, p

True

In [6]:
print(len(total_rows))
print(total_rows['92446699'])

749
{'country': 'Singapore', 'hasTakeaway': False, 'avgPrice': 200.0, 'rating': 5.0, 'telephones': ['+65-65601249'], 'lon': '103.84493759999998', 'cityId': '2311', 'geoPoint': {'lon': 103.84493759999998, 'lat': 1.281938}, 'title': '可楽伴手礼', 'honorCommentCount': 0, 'enviRating': 9.2, 'servRating': 9.2, 'lastCommentDate': '2018-10-08 12:37:07', 'qualRating': 9.0, 'branchTitle': '牛车水店', 'idVerified': False, 'state': 'Central', 'lat': '1.281938', 'recommendDishes': [{'name': '黄金凤梨球', 'value': 29}, {'name': '传统凤梨挞', 'value': 16}, {'name': '黄梨塔', 'value': 8}, {'name': '山王榴莲蛋糕卷', 'value': 7}, {'name': '抹茶凤梨球', 'value': 4}, {'name': '班蓝蛋糕', 'value': 4}, {'name': '杏仁饼干', 'value': 2}, {'name': '千层蛋糕', 'value': 2}, {'name': '虾米卷', 'value': 1}, {'name': '样品', 'value': 0}, {'name': 'cookies', 'value': 0}, {'name': '翡翠凤梨球', 'value': 0}], 'hasBooking': False, 'tags-inner': '服务热情,味道赞,性价比高,价格实惠,点心好', 'address': '2 Smith St Singapore 058917 Smith St, Singapore 058917', 'cityPinyin': 'singapore', 'catName

## Log

In [1]:
from utils import Log

LOG DIR: C:\Users\Evi\logs\analysis-tools\parrot.log


In [2]:
Log.info("testcases", "messages")

2019-06-08 13:28:28,815 INFO (testcases) messages


## dicts

In [3]:
from utils import dicts

In [5]:
counter = {}
dicts.add(counter, "word", 3)
print(counter)
dicts.add(counter, "word", 2)
print(counter)
dicts.add(counter, "word", 1)
print(counter)


{'word': 3}
{'word': 5}
{'word': 6}


## regexs

In [1]:
from utils import regexs

In [2]:
dir(regexs)

['SRE_MATCH_TYPE',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'chinese2digits',
 'chineseNumRegex',
 'chineseNumRegexRules',
 'convertChineseNum',
 'numeralMap',
 're']

In [3]:
print(regexs.chinese2digits(u"十三"))
print(regexs.convertChineseNum(u"十二稻"))
print(regexs.convertChineseNum(u"一万三稻"))

13.0
12.0稻
13000.0稻


## strings

In [7]:
from utils import strings

In [8]:
print(strings.stack_values(["id", "name", "age"]))

('%s,%s,%s', ['id', 'name', 'age'])
